# Feature Engineering for Properaty Appraisal and Comparable Recommendation System 🏠


## Overview
This notebook creates a system to find the top 3 comparable properties for real estate appraisal. Given a subject property, we'll identify the most similar recently sold properties that an appraiser would use for valuation.

## 1. Data Loading & Initial Setup 📊

**Purpose:** Load our cleaned dataset and establish the foundation for comparable property analysis.

**Appraisal Context:**
- 9,820 sold properties that can serve as comparables
- 88 subject properties that need appraisal
- Goal: For each subject, find 3 best comparable sales

**Comparable Selection Criteria (Industry Standard):**
- Similar size (GLA within ±20%)
- Same property type (detached, townhouse, etc.)
- Same neighborhood/city when possible
- Recent sales (within 90 days preferred)
- Similar age and condition

In [1]:
# Comparable Property Recommendation System for Appraisal
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances
from geopy.distance import geodesic
import warnings
warnings.filterwarnings('ignore')

# Load the cleaned dataset
df = pd.read_csv('data/processed/properties_deduplicated.csv')
subjects_df = pd.read_csv('data/processed/subjects_cleaned.csv')

print(f"📊 Dataset: {df.shape[0]:,} potential comparable properties")
print(f"🏠 {df.subject_id.nunique()} subject properties needing appraisal")
print(f"📋 Avg potential comps per subject: {df.groupby('subject_id').size().mean():.1f}")

# Basic data overview
print(f"\n🔍 Data Overview:")
print(f"   Price range: ${df['close_price'].min():,.0f} - ${df['close_price'].max():,.0f}")
print(f"   Size range: {df['gla_sqft'].min():.0f} - {df['gla_sqft'].max():.0f} sqft")
print(f"   Sale dates: {df['close_date'].min()} to {df['close_date'].max()}")

📊 Dataset: 7,246 potential comparable properties
🏠 88 subject properties needing appraisal
📋 Avg potential comps per subject: 82.3

🔍 Data Overview:
   Price range: $0 - $8,775,000
   Size range: 250 - 6930 sqft
   Sale dates: 2024-10-16 to 2025-05-13


## 2. Physical Property Similarity Features 🏗️

**Purpose:** Create features that measure physical similarity between subject properties and potential comparables - the foundation of appraisal methodology.

**Appraisal Standards We're Following:**
- **Size similarity**: GLA within ±20% is preferred, ±30% acceptable
- **Bedroom/bathroom similarity**: Exact match preferred, ±1 acceptable  
- **Structure type**: Must match (detached vs. townhouse affects value significantly)
- **Age similarity**: Properties within similar age brackets

**Why This Matters for Appraisal:**
- Physical characteristics are the primary drivers of property value
- Appraisers must justify why chosen comparables are truly "comparable"
- These features will be weighted heavily in our recommendation algorithm

In [2]:
def create_physical_similarity_features(df, subjects_df):
    """Create features measuring physical similarity for appraisal comparables"""
    print("🏗️ Creating physical similarity features for appraisal...")
    
    # Clean and prepare subject data
    subjects_clean = subjects_df.copy()
    
    # Convert subject bedrooms to numeric
    subjects_clean['bedrooms_clean'] = pd.to_numeric(subjects_clean['bedrooms_raw'], errors='coerce')
    
    # Parse subject bathrooms (format: "2:1" = 2 full + 1 half = 2.5 total)
    def parse_bathrooms(bath_str):
        if pd.isna(bath_str) or bath_str == '':
            return np.nan
        
        bath_str = str(bath_str).strip()
        
        # Handle "2:1" format (full:half)
        if ':' in bath_str:
            try:
                full, half = bath_str.split(':')
                return float(full) + float(half) * 0.5
            except:
                return np.nan
        
        # Handle "2F 1H" format  
        if 'F' in bath_str and 'H' in bath_str:
            try:
                parts = bath_str.replace('F', '').replace('H', '').split()
                if len(parts) == 2:
                    return float(parts[0]) + float(parts[1]) * 0.5
            except:
                return np.nan
        
        # Direct conversion
        try:
            return float(bath_str)
        except:
            return np.nan
    
    subjects_clean['bathrooms_clean'] = subjects_clean['bathrooms_raw'].apply(parse_bathrooms)
    
    # Merge subject characteristics with all properties
    df_with_subjects = df.merge(
        subjects_clean[['subject_id', 'gla_sqft', 'bedrooms_clean', 'bathrooms_clean', 
                       'structure_type', 'age_years']], 
        on='subject_id', 
        suffixes=('_prop', '_subj'),
        how='left'
    )
    
    # 1. SIZE SIMILARITY (Critical for appraisal)
    df['gla_diff_sqft'] = abs(df_with_subjects['gla_sqft_prop'] - df_with_subjects['gla_sqft_subj'])
    df['gla_diff_pct'] = df['gla_diff_sqft'] / df_with_subjects['gla_sqft_subj'] * 100
    
    # Appraisal size categories
    df['size_match_excellent'] = (df['gla_diff_pct'] <= 10).astype(int)  # Within 10%
    df['size_match_good'] = (df['gla_diff_pct'] <= 20).astype(int)       # Within 20%
    df['size_match_acceptable'] = (df['gla_diff_pct'] <= 30).astype(int) # Within 30%
    
    # 2. BEDROOM SIMILARITY
    df['bedroom_diff'] = abs(df['bedrooms_total'] - df_with_subjects['bedrooms_clean'])
    df['bedroom_exact_match'] = (df['bedroom_diff'] == 0).astype(int)
    df['bedroom_close_match'] = (df['bedroom_diff'] <= 1).astype(int)
    
    # 3. BATHROOM SIMILARITY  
    df['bathroom_diff'] = abs(df['bathrooms_equivalent'] - df_with_subjects['bathrooms_clean'])
    df['bathroom_exact_match'] = (df['bathroom_diff'] == 0).astype(int)
    df['bathroom_close_match'] = (df['bathroom_diff'] <= 0.5).astype(int)
    
    # 4. STRUCTURE TYPE MATCH (Critical - must match for good comp)
    df['structure_type_match'] = (df['structure_type'] == df_with_subjects['structure_type_subj']).astype(int)
    
    # 5. AGE SIMILARITY
    df['property_age'] = 2025 - df['year_built']
    df['age_diff_years'] = abs(df['property_age'] - df_with_subjects['age_years'])
    df['age_similar'] = (df['age_diff_years'] <= 10).astype(int)  # Within 10 years
    
    print(f"✅ Physical similarity features created:")
    print(f"   Size excellent match (≤10%): {df['size_match_excellent'].sum():,}/{len(df):,} ({df['size_match_excellent'].mean()*100:.1f}%)")
    print(f"   Size good match (≤20%): {df['size_match_good'].sum():,}/{len(df):,} ({df['size_match_good'].mean()*100:.1f}%)")
    print(f"   Bedroom exact match: {df['bedroom_exact_match'].sum():,}/{len(df):,} ({df['bedroom_exact_match'].mean()*100:.1f}%)")
    print(f"   Bathroom exact match: {df['bathroom_exact_match'].sum():,}/{len(df):,} ({df['bathroom_exact_match'].mean()*100:.1f}%)")
    print(f"   Structure type match: {df['structure_type_match'].sum():,}/{len(df):,} ({df['structure_type_match'].mean()*100:.1f}%)")
    
    return df

# Run the function
df = create_physical_similarity_features(df, subjects_df)

🏗️ Creating physical similarity features for appraisal...
✅ Physical similarity features created:
   Size excellent match (≤10%): 959/7,246 (13.2%)
   Size good match (≤20%): 1,994/7,246 (27.5%)
   Bedroom exact match: 2,390/7,246 (33.0%)
   Bathroom exact match: 589/7,246 (8.1%)
   Structure type match: 1,110/7,246 (15.3%)


## 3. Location & Proximity Features 🌍


**Purpose:** Create location-based features that capture geographic proximity and market area similarity - critical for appraisal validity.

**Appraisal Location Standards:**
- **Same neighborhood preferred**: FSA (postal code area) matching
- **Same city acceptable**: When neighborhood comps are limited
- **Distance matters**: Closer properties are more comparable
- **Market area consistency**: Properties should be in similar market conditions

**Why Location is Critical for Appraisal:**
- Location is the #1 factor affecting property value
- Appraisers must justify geographic proximity of comparables
- Different neighborhoods can have significantly different price per sqft
- Distance adjustments may be needed for remote comparables

In [3]:
def create_location_features(df, subjects_df):
    """Create location-based features for appraisal comparables"""
    print("🌍 Creating location features for appraisal...")
    
    # 1. CITY/MUNICIPALITY MATCHING  
    df['same_city'] = (df['city'] == df['city'].groupby(df['subject_id']).transform('first')).astype(int)
    df['same_province'] = (df['state_province'] == df['state_province'].groupby(df['subject_id']).transform('first')).astype(int)
    
    # 2. POSTAL CODE AREA MATCHING (FSA = first 3 digits)
    df['postal_fsa'] = df['postal_code'].str[:3]
    df['subject_fsa'] = df['postal_fsa'].groupby(df['subject_id']).transform('first')
    df['same_fsa'] = (df['postal_fsa'] == df['subject_fsa']).astype(int)
    
    # 3. DISTANCE CALCULATION
    def calc_distance_to_subject(group):
        if len(group) == 0:
            return pd.Series(dtype=float)
            
        # Use first property as subject location proxy
        subj_lat = group['latitude'].iloc[0]
        subj_lon = group['longitude'].iloc[0]
        
        distances = []
        for _, row in group.iterrows():
            if pd.notna(row['latitude']) and pd.notna(row['longitude']) and \
               pd.notna(subj_lat) and pd.notna(subj_lon):
                try:
                    dist = geodesic((subj_lat, subj_lon), 
                                  (row['latitude'], row['longitude'])).kilometers
                    distances.append(dist)
                except:
                    distances.append(np.nan)
            else:
                distances.append(np.nan)
        
        return pd.Series(distances, index=group.index)
    
    df['distance_km'] = df.groupby('subject_id').apply(calc_distance_to_subject).values
    
    # 4. DISTANCE CATEGORIES FOR APPRAISAL
    df['distance_excellent'] = (df['distance_km'] <= 1).astype(int)    # Within 1km
    df['distance_good'] = (df['distance_km'] <= 5).astype(int)         # Within 5km  
    df['distance_acceptable'] = (df['distance_km'] <= 15).astype(int)  # Within 15km
    
    print(f"✅ Location features created:")
    print(f"   Same city: {df['same_city'].sum():,}/{len(df):,} ({df['same_city'].mean()*100:.1f}%)")
    print(f"   Same FSA: {df['same_fsa'].sum():,}/{len(df):,} ({df['same_fsa'].mean()*100:.1f}%)")
    print(f"   Distance ≤1km: {df['distance_excellent'].sum():,}/{len(df):,} ({df['distance_excellent'].mean()*100:.1f}%)")
    print(f"   Distance ≤5km: {df['distance_good'].sum():,}/{len(df):,} ({df['distance_good'].mean()*100:.1f}%)")
    print(f"   Avg distance: {df['distance_km'].mean():.1f}km")
    
    return df

# Run the function
df = create_location_features(df, subjects_df)

🌍 Creating location features for appraisal...
✅ Location features created:
   Same city: 6,151/7,246 (84.9%)
   Same FSA: 4,273/7,246 (59.0%)
   Distance ≤1km: 2,265/7,246 (31.3%)
   Distance ≤5km: 6,848/7,246 (94.5%)
   Avg distance: 2.0km


## 4. Temporal Features ⏰



**Purpose:** Create features that measure how recent the comparable sales are - fresher sales are more reliable for current market valuation.

**Appraisal Temporal Standards:**
- **≤90 days preferred**: Most current market conditions
- **≤180 days acceptable**: Still relevant for most markets
- **>6 months**: May need market condition adjustments
- **Sale date vs. effective date**: Compare to subject's effective date, not today

**Why Recency Matters for Appraisal:**
- Market conditions change rapidly (especially in volatile markets)
- Appraisers prefer recent sales to reflect current market value
- Older sales may require time adjustments
- Seasonal effects can impact comparability

In [4]:
def create_temporal_features(df, subjects_df):
    """Create temporal features measuring sale recency for appraisal"""
    print("⏰ Creating temporal features for appraisal...")
    
    # Convert dates to datetime
    df['close_date'] = pd.to_datetime(df['close_date'])
    subjects_df['effective_date'] = pd.to_datetime(subjects_df['effective_date'])
    
    # Merge subject effective dates
    df_with_subjects = df.merge(
        subjects_df[['subject_id', 'effective_date']], 
        on='subject_id', 
        how='left'
    )
    
    # 1. DAYS BETWEEN SALE AND APPRAISAL
    df['days_from_effective'] = (df_with_subjects['effective_date'] - df['close_date']).dt.days
    
    # 2. RECENCY CATEGORIES (Appraisal Standards)
    df['sale_very_recent'] = (df['days_from_effective'] <= 90).astype(int)    # ≤3 months
    df['sale_recent'] = (df['days_from_effective'] <= 180).astype(int)        # ≤6 months  
    df['sale_acceptable'] = (df['days_from_effective'] <= 365).astype(int)    # ≤1 year
    
    # 3. RECENCY SCORE (0-1, higher = more recent)
    max_days = df['days_from_effective'].max()
    df['recency_score'] = 1 - (df['days_from_effective'] / max_days)
    df['recency_score'] = df['recency_score'].clip(0, 1)
    
    # 4. SEASONAL CONSIDERATIONS
    df['sale_month'] = df['close_date'].dt.month
    df['effective_month'] = df_with_subjects['effective_date'].dt.month
    df['same_season'] = (
        ((df['sale_month'].isin([12, 1, 2])) & (df['effective_month'].isin([12, 1, 2]))) |  # Winter
        ((df['sale_month'].isin([3, 4, 5])) & (df['effective_month'].isin([3, 4, 5]))) |    # Spring
        ((df['sale_month'].isin([6, 7, 8])) & (df['effective_month'].isin([6, 7, 8]))) |    # Summer
        ((df['sale_month'].isin([9, 10, 11])) & (df['effective_month'].isin([9, 10, 11])))  # Fall
    ).astype(int)
    
    print(f"✅ Temporal features created:")
    print(f"   Sales ≤90 days: {df['sale_very_recent'].sum():,}/{len(df):,} ({df['sale_very_recent'].mean()*100:.1f}%)")
    print(f"   Sales ≤180 days: {df['sale_recent'].sum():,}/{len(df):,} ({df['sale_recent'].mean()*100:.1f}%)")
    print(f"   Sales ≤1 year: {df['sale_acceptable'].sum():,}/{len(df):,} ({df['sale_acceptable'].mean()*100:.1f}%)")
    print(f"   Same season: {df['same_season'].sum():,}/{len(df):,} ({df['same_season'].mean()*100:.1f}%)")
    print(f"   Avg days from effective: {df['days_from_effective'].mean():.0f} days")
    
    return df

# Run the function
df = create_temporal_features(df, subjects_df)

⏰ Creating temporal features for appraisal...
✅ Temporal features created:
   Sales ≤90 days: 6,910/7,246 (95.4%)
   Sales ≤180 days: 7,246/7,246 (100.0%)
   Sales ≤1 year: 7,246/7,246 (100.0%)
   Same season: 4,483/7,246 (61.9%)
   Avg days from effective: 33 days


## 5. Market & Price Features 💰

**Purpose:** Create features that capture market dynamics and price relationships - essential for understanding value patterns and market positioning.

**Appraisal Market Analysis Standards:**
- **Price per sqft comparison**: Core metric for size-adjusted value
- **Market position**: How property compares to local market (above/below average)
- **Price range compatibility**: Properties should be in similar value brackets
- **Market trend awareness**: Understanding if market is rising/falling

**Why Market Features Matter for Appraisal:**
- Price per sqft is the primary adjustment metric appraisers use
- Properties in different price tiers may not be truly comparable
- Market trends affect how recent sales should be interpreted
- Helps identify outliers that may not be good comparables

In [5]:
def create_market_price_features(df, subjects_df):
    """Create market and price-based features for appraisal comparables"""
    print("💰 Creating market & price features for appraisal...")
    
    # 1. PRICE PER SQUARE FOOT ANALYSIS
    df['price_per_sqft'] = df['close_price'] / df['gla_sqft']
    
    # Calculate subject estimated price per sqft (using median of similar properties)
    def calc_subject_price_per_sqft(group):
        # Use median price/sqft of similar sized properties as proxy
        similar_size = group[abs(group - group.iloc[0]) <= 50]  # Within 50 $/sqft
        if len(similar_size) > 0:
            return similar_size.median()
        else:
            return group.median()
    
    df['subject_price_per_sqft_est'] = df.groupby('subject_id')['price_per_sqft'].transform(calc_subject_price_per_sqft)
    
    # 2. PRICE PER SQFT SIMILARITY
    df['price_per_sqft_diff'] = abs(df['price_per_sqft'] - df['subject_price_per_sqft_est'])
    df['price_per_sqft_diff_pct'] = (df['price_per_sqft_diff'] / df['subject_price_per_sqft_est']) * 100
    
    # Price per sqft categories
    df['price_psf_very_similar'] = (df['price_per_sqft_diff_pct'] <= 10).astype(int)  # Within 10%
    df['price_psf_similar'] = (df['price_per_sqft_diff_pct'] <= 20).astype(int)       # Within 20%
    df['price_psf_acceptable'] = (df['price_per_sqft_diff_pct'] <= 30).astype(int)    # Within 30%
    
    # 3. MARKET POSITION FEATURES
    # Calculate market percentiles by city
    df['city_price_percentile'] = df.groupby('city')['close_price'].rank(pct=True)
    df['city_psf_percentile'] = df.groupby('city')['price_per_sqft'].rank(pct=True)
    
    # Market tier classification
    df['market_tier'] = pd.cut(df['city_price_percentile'], 
                              bins=[0, 0.33, 0.67, 1.0], 
                              labels=['Lower', 'Middle', 'Upper'])
    
    # Same market tier as subject
    df['subject_market_tier'] = df.groupby('subject_id')['market_tier'].transform('first')
    df['same_market_tier'] = (df['market_tier'] == df['subject_market_tier']).astype(int)
    
    # 4. VALUE ADJUSTMENT INDICATORS
    df['needs_size_adjustment'] = (df['gla_diff_pct'] > 10).astype(int)
    df['needs_location_adjustment'] = (df['same_city'] == 0).astype(int)
    df['needs_time_adjustment'] = (df['days_from_effective'] > 90).astype(int)
    
    # Total adjustments needed (fewer is better for comparables)
    df['total_adjustments_needed'] = (df['needs_size_adjustment'] + 
                                     df['needs_location_adjustment'] + 
                                     df['needs_time_adjustment'])
    
    print(f"✅ Market & price features created:")
    print(f"   Price/sqft very similar (≤10%): {df['price_psf_very_similar'].sum():,}/{len(df):,} ({df['price_psf_very_similar'].mean()*100:.1f}%)")
    print(f"   Same market tier: {df['same_market_tier'].sum():,}/{len(df):,} ({df['same_market_tier'].mean()*100:.1f}%)")
    print(f"   No adjustments needed: {(df['total_adjustments_needed'] == 0).sum():,}/{len(df):,} ({(df['total_adjustments_needed'] == 0).mean()*100:.1f}%)")
    print(f"   Avg price/sqft: ${df['price_per_sqft'].mean():.0f}")
    
    return df

# Run the function
df = create_market_price_features(df, subjects_df)

💰 Creating market & price features for appraisal...
✅ Market & price features created:
   Price/sqft very similar (≤10%): 2,129/7,246 (29.4%)
   Same market tier: 2,894/7,246 (39.9%)
   No adjustments needed: 835/7,246 (11.5%)
   Avg price/sqft: $513


## 6. Composite Comparable Score & Ranking 🎯

**Purpose:** Create a comprehensive scoring system that combines all similarity features to rank potential comparables - mimicking how appraisers evaluate and select the best comparables.

**Appraisal Scoring Methodology:**
- **Physical similarity (40%)**: Size, bedrooms, bathrooms, structure type
- **Location proximity (30%)**: Distance, same neighborhood/city
- **Temporal recency (20%)**: How recent the sale is
- **Market compatibility (10%)**: Price range and market tier alignment

**Final Output:**
- Composite score (0-100) for each potential comparable
- Ranking within each subject property group
- Top 3 recommendations per subject (ready for appraisal use)
- Quality flags for exceptional vs. marginal comparables

**Why This Matters:**
- Provides objective, defensible comparable selection
- Ensures consistency across different appraisers
- Identifies when comparable supply is limited (quality warnings)
- Creates audit trail for appraisal review process

In [6]:
def create_composite_comparable_score(df):
    """Create comprehensive scoring system for comparable property ranking"""
    print("🎯 Creating composite comparable scoring system...")
    
    # WEIGHT CONFIGURATION (Based on Appraisal Standards)
    weights = {
        'physical': 0.40,    # Size, bedrooms, bathrooms, structure type
        'location': 0.30,    # Distance, same neighborhood/city  
        'temporal': 0.20,    # Sale recency
        'market': 0.10       # Price compatibility, market tier
    }
    
    # 1. PHYSICAL SIMILARITY SCORE (0-100)
    physical_score = (
        df['size_match_excellent'] * 30 +           # Perfect size match
        df['size_match_good'] * 20 +                # Good size match
        df['size_match_acceptable'] * 10 +          # Acceptable size match
        df['bedroom_exact_match'] * 25 +            # Bedroom match
        df['bathroom_exact_match'] * 15 +           # Bathroom match
        df['structure_type_match'] * 30 +           # Structure type (critical)
        df['age_similar'] * 10                      # Age similarity
    )
    df['physical_score'] = physical_score.clip(0, 100)
    
    # 2. LOCATION PROXIMITY SCORE (0-100)
    location_score = (
        df['same_fsa'] * 40 +                       # Same postal area (best)
        df['same_city'] * 25 +                      # Same city
        df['distance_excellent'] * 30 +            # Within 1km
        df['distance_good'] * 15 +                  # Within 5km
        df['distance_acceptable'] * 5               # Within 15km
    )
    df['location_score'] = location_score.clip(0, 100)
    
    # 3. TEMPORAL RECENCY SCORE (0-100)
    temporal_score = (
        df['sale_very_recent'] * 50 +               # ≤90 days (preferred)
        df['sale_recent'] * 30 +                    # ≤180 days
        df['sale_acceptable'] * 15 +                # ≤1 year
        df['same_season'] * 20 +                    # Seasonal consistency
        df['recency_score'] * 30                    # Continuous recency score
    )
    df['temporal_score'] = temporal_score.clip(0, 100)
    
    # 4. MARKET COMPATIBILITY SCORE (0-100)
    market_score = (
        df['price_psf_very_similar'] * 40 +         # Price/sqft very similar
        df['price_psf_similar'] * 25 +              # Price/sqft similar
        df['price_psf_acceptable'] * 15 +           # Price/sqft acceptable
        df['same_market_tier'] * 30 +               # Same market tier
        (3 - df['total_adjustments_needed']) * 10   # Fewer adjustments needed
    )
    df['market_score'] = market_score.clip(0, 100)
    
    # 5. COMPOSITE SCORE (Weighted Average)
    df['composite_score'] = (
        df['physical_score'] * weights['physical'] +
        df['location_score'] * weights['location'] +
        df['temporal_score'] * weights['temporal'] +
        df['market_score'] * weights['market']
    )
    
    # 6. RANKING WITHIN EACH SUBJECT
    df['comparable_rank'] = df.groupby('subject_id')['composite_score'].rank(
        method='dense', ascending=False
    ).astype(int)
    
    # 7. QUALITY CLASSIFICATION
    def classify_comparable_quality(score):
        if score >= 80:
            return 'Excellent'
        elif score >= 60:
            return 'Good'
        elif score >= 40:
            return 'Fair'
        else:
            return 'Poor'
    
    df['comparable_quality'] = df['composite_score'].apply(classify_comparable_quality)
    
    # 8. TOP 3 RECOMMENDATIONS PER SUBJECT
    df['is_top3_comparable'] = (df['comparable_rank'] <= 3).astype(int)
    
    print(f"✅ Composite scoring completed:")
    print(f"   Excellent comparables (≥80): {(df['comparable_quality'] == 'Excellent').sum():,} ({(df['comparable_quality'] == 'Excellent').mean()*100:.1f}%)")
    print(f"   Good comparables (≥60): {(df['comparable_quality'] == 'Good').sum():,} ({(df['comparable_quality'] == 'Good').mean()*100:.1f}%)")
    print(f"   Top 3 selections: {df['is_top3_comparable'].sum():,} total recommendations")
    print(f"   Avg composite score: {df['composite_score'].mean():.1f}")
    
    # Quality check per subject
    top3_quality = df[df['is_top3_comparable'] == 1]['comparable_quality'].value_counts()
    print(f"\n📊 Top 3 Comparable Quality Distribution:")
    for quality, count in top3_quality.items():
        print(f"   {quality}: {count} ({count/df['is_top3_comparable'].sum()*100:.1f}%)")
    
    return df

# Run the function
df = create_composite_comparable_score(df)

🎯 Creating composite comparable scoring system...
✅ Composite scoring completed:
   Excellent comparables (≥80): 704 (9.7%)
   Good comparables (≥60): 2,298 (31.7%)
   Top 3 selections: 449 total recommendations
   Avg composite score: 57.6

📊 Top 3 Comparable Quality Distribution:
   Excellent: 322 (71.7%)
   Good: 95 (21.2%)
   Fair: 32 (7.1%)


In [7]:
# ## 8. Fix Top 3 Recommendations (Proper Tie-Breaking) 🔧

# **Purpose:** Fix the ranking issue where ties in composite scores caused some subjects to have more or fewer than 3 recommendations.
# 
# **Problem:** When multiple properties had identical composite scores, they all received the same rank, causing inconsistent top 3 selections.
# 
# **Solution:** Implement proper tie-breaking using secondary criteria (distance, recency) to ensure exactly 3 recommendations per subject.

print("🔧 FIXING TOP 3 RECOMMENDATIONS WITH PROPER TIE-BREAKING")
print("=" * 60)

# Current issue analysis
current_top3_counts = df[df['is_top3_comparable'] == 1].groupby('subject_id').size()
print(f"📊 CURRENT ISSUE ANALYSIS:")
print(f"   Recommendations per subject distribution:")
print(f"   {current_top3_counts.value_counts().sort_index()}")
print(f"   Subjects with exactly 3 recommendations: {(current_top3_counts == 3).sum()}/{len(current_top3_counts)}")

# Fix ranking with proper tie-breaking
def rank_comparables_properly(group):
    """Rank comparables with proper tie-breaking using multiple criteria"""
    # Sort by:
    # 1. Composite score (descending - higher is better)
    # 2. Distance (ascending - closer is better) 
    # 3. Recency score (descending - more recent is better)
    # 4. Property ID (ascending - for final tie-breaking)
    group_sorted = group.sort_values([
        'composite_score', 
        'distance_km', 
        'recency_score',
        'property_id'
    ], ascending=[False, True, False, True])
    
    # Assign sequential ranks (1, 2, 3, ...)
    group_sorted['comparable_rank_fixed'] = range(1, len(group_sorted) + 1)
    
    return group_sorted

print(f"\n🔄 APPLYING PROPER RANKING...")

# Apply proper ranking to each subject group
df_fixed = df.groupby('subject_id').apply(rank_comparables_properly).reset_index(drop=True)

# Create new top 3 flag with fixed ranking
df_fixed['is_top3_comparable_fixed'] = (df_fixed['comparable_rank_fixed'] <= 3).astype(int)

# Verify the fix
fixed_top3_counts = df_fixed[df_fixed['is_top3_comparable_fixed'] == 1].groupby('subject_id').size()

print(f"\n✅ FIXED RESULTS:")
print(f"   Recommendations per subject distribution:")
print(f"   {fixed_top3_counts.value_counts().sort_index()}")
print(f"   Subjects with exactly 3 recommendations: {(fixed_top3_counts == 3).sum()}/{len(fixed_top3_counts)}")

# Handle edge cases (subjects with < 3 available properties)
subjects_with_few_props = fixed_top3_counts[fixed_top3_counts < 3]
if len(subjects_with_few_props) > 0:
    print(f"\n⚠️  EDGE CASES - Subjects with < 3 available properties:")
    for subject_id, count in subjects_with_few_props.items():
        total_available = len(df_fixed[df_fixed['subject_id'] == subject_id])
        print(f"   Subject {subject_id}: {count} recommendations (only {total_available} properties available)")

# Update the main dataframe
df = df_fixed.copy()

# Create clean top 3 dataset
top3_comparables_fixed = df[df['is_top3_comparable_fixed'] == 1].copy()

print(f"\n📊 FINAL SUMMARY:")
print(f"   Total properties: {len(df):,}")
print(f"   Total top 3 recommendations: {len(top3_comparables_fixed):,}")
print(f"   Average composite score of top 3: {top3_comparables_fixed['composite_score'].mean():.1f}")

# Quality distribution of fixed top 3
print(f"\n🏆 FIXED TOP 3 QUALITY DISTRIBUTION:")
quality_dist_fixed = top3_comparables_fixed['comparable_quality'].value_counts()
for quality in ['Excellent', 'Good', 'Fair', 'Poor']:
    if quality in quality_dist_fixed.index:
        count = quality_dist_fixed[quality]
        pct = count / len(top3_comparables_fixed) * 100
        print(f"   {quality}: {count} ({pct:.1f}%)")

print(f"\n💾 READY TO SAVE:")
print(f"   • Fixed dataset: {len(df):,} rows × {len(df.columns)} columns")
print(f"   • Fixed top 3 recommendations: {len(top3_comparables_fixed):,} rows") 

🔧 FIXING TOP 3 RECOMMENDATIONS WITH PROPER TIE-BREAKING
📊 CURRENT ISSUE ANALYSIS:
   Recommendations per subject distribution:
   3     33
4     19
5     12
6     12
7      3
8      1
10     1
11     1
12     1
13     2
16     1
17     1
21     1
Name: count, dtype: int64
   Subjects with exactly 3 recommendations: 33/88

🔄 APPLYING PROPER RANKING...

✅ FIXED RESULTS:
   Recommendations per subject distribution:
   3    88
Name: count, dtype: int64
   Subjects with exactly 3 recommendations: 88/88

📊 FINAL SUMMARY:
   Total properties: 7,246
   Total top 3 recommendations: 264
   Average composite score of top 3: 81.4

🏆 FIXED TOP 3 QUALITY DISTRIBUTION:
   Excellent: 169 (64.0%)
   Good: 71 (26.9%)
   Fair: 24 (9.1%)

💾 READY TO SAVE:
   • Fixed dataset: 7,246 rows × 92 columns
   • Fixed top 3 recommendations: 264 rows


## 7. Save Engineered Dataset & Summary 💾

**Purpose:** Save the fully engineered dataset and provide a comprehensive summary of our comparable recommendation system.

**What We've Built:**
- Complete feature engineering for appraisal-grade comparable selection
- 40+ engineered features covering physical, location, temporal, and market similarity
- Composite scoring system with industry-standard weighting
- Top 3 comparable recommendations per subject property

**Dataset Output:**
- Ready for statistical analysis or machine learning model training
- Each row represents a potential comparable with full similarity scoring
- Top 3 recommendations clearly flagged for immediate appraisal use
- Quality classifications for risk assessment

In [8]:
# Save the fully engineered dataset
print("💾 Saving engineered dataset...")

# Save complete dataset with all features
df.to_csv('data/processed/properties_comparison_engineered.csv', index=False)

💾 Saving engineered dataset...
✅ Datasets saved:
   Complete dataset: 7,246 rows × 92 columns
   Top 3 recommendations: 449 rows

🏠 COMPARABLE RECOMMENDATION SYSTEM SUMMARY

📊 DATASET OVERVIEW:
   • 88 subject properties analyzed
   • 7,246 potential comparable properties
   • 92 total features engineered
   • 449 top recommendations selected

🎯 FEATURE ENGINEERING RESULTS:
   Physical Similarity:
     - Size excellent match: 13.2%
     - Structure type match: 15.3%
     - Bedroom exact match: 33.0%
   Location Proximity:
     - Same city: 84.9%
     - Same FSA: 59.0%
     - Within 1km: 31.3%
   Temporal Recency:
     - Sales ≤90 days: 95.4%
     - Sales ≤180 days: 100.0%
   Market Compatibility:
     - Price/sqft similar: 49.7%
     - Same market tier: 39.9%

🏆 RECOMMENDATION QUALITY:
   Excellent: 9.7%
   Good: 31.7%
   Fair: 44.2%
   Poor: 14.4%

📈 TOP 3 RECOMMENDATIONS PER SUBJECT:
   Average score of top 3: 83.3
   Subjects with 3 excellent comps: 8/88
   Subjects with 0 excellent